In [ ]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragragh Text. 
#Assign the text to variables that you can reference later.
#Visit the url for JPL's Featured Space Image here.
#Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign 
#the url string to a variable called featured_image_url.

In [8]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import pandas as pd



In [9]:
# Initialize PyMongo to work with MongoDBs, make sure in terminal, fire up mongoDB by typing "mongod"
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [11]:
# Define database and collection
db = client.mars_db
collection = db.newstitle

In [649]:
#url to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

In [650]:
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [659]:
# print(soup)
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all(class_="slide")
print(results)

[<div class="slide">
<div class="image_and_description_container">
<a href="/news/8326/nasa-invests-in-visionary-technology/">
<div class="rollover_description">
<div class="rollover_description_inner">
NASA is investing in technology concepts, including several from JPL, that may one day be used for future space exploration missions.
</div>
<div class="overlay_arrow">
<img alt="More" src="/assets/overlay-arrow.png"/>
</div>
</div>
<img alt="NASA Invests in Visionary Technology " class="img-lazy" data-lazy="/system/news_items/list_view_images/8326_niac320.jpg" src="/assets/loading_320x240.png"/>
</a>
</div>
<div class="content_title">
<a href="/news/8326/nasa-invests-in-visionary-technology/">
NASA Invests in Visionary Technology 
</a>
</div>
</div>, <div class="slide">
<div class="image_and_description_container">
<a href="/news/8325/nasa-is-ready-to-study-the-heart-of-mars/">
<div class="rollover_description">
<div class="rollover_description_inner">
NASA is about to go on a journey 

In [654]:
# Loop through returned results
for result in results:
    # Error handling
    try:
        
        
        paragraph = result.find(class_="rollover_description_inner").text.replace('\n', '')
        title = result.find(class_="content_title").text.replace('\n', '')
        
        # Print results only if title, price, and link are available
        if (paragraph and title):
            print('-------------')
            print(title)
            print(paragraph)
            
            
            #Dictionary to be inserted as a MongoDb document
            post = {"title": title,
                    "paragraph":paragraph}
            
    except Exception as e:
           print(e)
        

-------------
NASA Invests in Visionary Technology 
NASA is investing in technology concepts, including several from JPL, that may one day be used for future space exploration missions.
-------------
NASA is Ready to Study the Heart of Mars
NASA is about to go on a journey to study the center of Mars.
-------------
NASA Briefing on First Mission to Study Mars Interior
NASA’s next mission to Mars will be the topic of a media briefing Thursday, March 29, at JPL. The briefing will air live on NASA Television and the agency’s website.
-------------
New 'AR' Mobile App Features 3-D NASA Spacecraft
NASA spacecraft travel to far-off destinations in space, but a new mobile app produced by NASA's Jet Propulsion Laboratory, Pasadena, California, brings spacecraft to users.
-------------
Witness First Mars Launch from West Coast
NASA invites digital creators to apply for social media credentials to cover the launch of the InSight mission to Mars, May 3-5, at California's Vandenberg Air Force Base

In [492]:
#Visit the url for JPL's Featured Space Image here.
#Use splinter to navigate the site and find the image url for the current Featured Mars Image 
#and assign the url string to a variable called featured_image_url.

browser = Browser('chrome', headless=False)
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [493]:
#click on element with partial text of 'FULL IMAGE', this is case sensitive
browser.click_link_by_partial_text('FULL IMAGE')

In [494]:
#HTML Object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
results = soup.find(class_='fancybox-image')
url = results['src']
featured_image_url = 'https://www.jpl.nasa.gov'+url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14317_ip.jpg


In [335]:
#visit the Mars Twitter Account to scrape latest weather tweet
#Save the tweet text for the weather report as a variable called mars_weather.
browser = Browser('chrome', headless=False)
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [336]:
#url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [337]:
#after inspecting site, return the first 'p' tag with class listed below to return latest weather tweet.
#.find gets just the first one (latest).find_all would return all of them...

mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
print(mars_weather)


Sol 2033 (April 25, 2018), Sunny, high -10C/14F, low -71C/-95F, pressure at 7.23 hPa, daylight 05:24-17:20


In [40]:
#visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet 
#including Diameter, Mass, etc.
url = 'https://space-facts.com/mars/'
#read the table, put into list variable
tables = pd.read_html(url)
#convert the list to a dataframe
mars_df =tables[0].replace("/n","")
#set column names 
mars_df.columns = ["Characteristics","Values"]
#set index to Characteristics
mars_df = mars_df.set_index(["Characteristics"])
#make it an html table
print(mars_df)

                                             Values
Characteristics                                    
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.42 x 10^23 kg (10.7% Earth)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.52 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                  -153 to 20 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astronomers


In [41]:
#Visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres.
#You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
#Save both the image url string for the full resolution hemipshere image, and the Hemisphere title 
#containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
#Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one 
#dictionary for each hemisphere.

In [352]:
browser = Browser('chrome', headless=False)
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [233]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
#return the data underneath the class 'item' tag
results = soup.find_all(class_='item')
print(results)


[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>, <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/7677c0a006b83871b5a2f66985ab5857_schiapar

In [252]:
mars_images=[]
for result in results:
    #reference the 'a' tags withing 'item' class
    link=result.find('a')
    #store the 'href', i.e. link in a variable 'links'
    links = link['href']
    #store the 'h3' tag text within item in a variable 'title'
    title =result.find('h3').text
    #attach the 'href' link from above to the main URL, so you can access the final page
    url='https://astrogeology.usgs.gov'+links
    browser = Browser('chrome', headless=False)
    #visit the new concatenated URL
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    #reference the 'downloads' class, store in a variable 'infos'
    infos = soup.find_all(class_='downloads')
    #loop thru 'infos' pulling the 'a' tags, 'href', where full resolution image is stored
    for info in infos:
        link_two=info.find('a')
        img_url=link_two['href']
    print(title)
    print(img_url)
    #create a dictionary of title and image url
    post={'img_url':img_url, 'title':title}
    #append the dictionary to list/array 'mars_images'
    mars_images.append(post)
    

Cerberus Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [253]:
mars_images

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [46]:
def init_browser():
    browser = Browser('chrome', headless=False)
    #return Browser("chrome", **executable_path, headless=False)


In [47]:
def scrape():
   
    browser = init_browser()
    url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
    response = requests.get(url)
    time.sleep(1)
    soup = BeautifulSoup(response.text, 'lxml')
    #grabbling the 'slide' class element from the url
    results = soup.find_all(class_="slide")
    #creating a list to hold scraped data
    news_data=[]
    for result in results:
    # Error handling
        try: #loop thru and get the text within these classes, replace \n with blank space
            news_p = result.find(class_="rollover_description_inner").text.replace('\n', '')
            news_title = result.find(class_="content_title").text.replace('\n', '')
           
            post = {"news_title": news_title,
                    "news_p":news_p}

            news_data.append(post)
            print(post)
          
        except Exception as e:
               print(e)
    browser = Browser('chrome', headless=False)
    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)
    time.sleep(1)
    #use splinter to click the "Full Image" button
    browser.click_link_by_partial_text('FULL IMAGE')
    time.sleep(1)
    #HTML Object 
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    #find the class where pic is stored
    results = soup.find(class_='fancybox-image')
    #retrieve source attribute, i.e. the path
    url = results['src']
    #attach the path to the main site link, this is the full image link
    featured_image_url = 'https://www.jpl.nasa.gov'+url
    post_two = {'featured_image':featured_image_url}
    news_data.append(post_two)
    print(post_two)
    #visit the mars twitter page to get the Weather
    url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(url)
    time.sleep(1)
    response = requests.get(url)
    #parse HTML with Beautiful soup, get the text
    soup = BeautifulSoup(response.text, 'html.parser')
    #get the text from the first p tag with appropriate class (from inspecting the site)
    mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
    post_three = {'mars_weather':mars_weather}
    print(post_three)
    news_data.append(post_three)
    browser = Browser('chrome', headless=False)
    #visit the mars space facts site
    url = 'https://space-facts.com/mars/'
    #read the table, put into list variable
    tables = pd.read_html(url)
    #convert the list to a dataframe
    mars_df =tables[0]
    #put column headers on 
    mars_df.columns = ["Characteristic", "Value"]
    #convert the datframe to dictionary, using 'records' orientation, this does not neeed to be, nor should be, appended to news_data, as will create a list of a dictionary within the list, and not be able to be inserted to mongodb
    mars_dict=mars_df.to_dict('records')
    print(mars_dict)
    #Visit the site to get images of Mars Hemispheres
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    time.sleep(1)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all(class_='item')
  
    #loop through the item class
    for result in results:
        #find the first a tag
        link=result.find('a')
        #assign the href to variable 'links'
        links = link['href']
        #assign the link h3 title text to variable 'title'
        title =result.find('h3').text
        #concatenate the path with the main site link, assign to variable 'url'
        url='https://astrogeology.usgs.gov'+links
        #open brower, chromedriver
        browser = Browser('chrome', headless=False)
        #visit the concatenated url
        browser.visit(url)
        time.sleep(1)
        html = browser.html
        #parse the html with beautiful soup
        soup = BeautifulSoup(html, 'html.parser')
        #find all elemenst with class 'downloads', assign results to variable list 'infos'
        infos = soup.find_all(class_='downloads')
        #loop thru infos, pull out links to images, assign with title to dictionary post, and then append to list
        #mars_images
        for info in infos:
            link_two=info.find('a')
            img_url=link_two['href']
            post_four={'img_url':img_url, 'title':title}
            news_data.append(post_four)
            print(post_four)
   
   #return your data, so it can be accessed by flask app (where the insertion into mongodb will occur)
    return news_data+mars_dict
    
            

In [48]:
scrape()

{'news_title': 'NASA Invests in Visionary Technology ', 'news_p': 'NASA is investing in technology concepts, including several from JPL, that may one day be used for future space exploration missions.'}
{'news_title': 'NASA is Ready to Study the Heart of Mars', 'news_p': 'NASA is about to go on a journey to study the center of Mars.'}
{'news_title': 'NASA Briefing on First Mission to Study Mars Interior', 'news_p': 'NASA’s next mission to Mars will be the topic of a media briefing Thursday, March 29, at JPL. The briefing will air live on NASA Television and the agency’s website.'}
{'news_title': "New 'AR' Mobile App Features 3-D NASA Spacecraft", 'news_p': "NASA spacecraft travel to far-off destinations in space, but a new mobile app produced by NASA's Jet Propulsion Laboratory, Pasadena, California, brings spacecraft to users."}
{'news_title': 'Witness First Mars Launch from West Coast', 'news_p': "NASA invites digital creators to apply for social media credentials to cover the launch

[{'news_p': 'NASA is investing in technology concepts, including several from JPL, that may one day be used for future space exploration missions.',
  'news_title': 'NASA Invests in Visionary Technology '},
 {'news_p': 'NASA is about to go on a journey to study the center of Mars.',
  'news_title': 'NASA is Ready to Study the Heart of Mars'},
 {'news_p': 'NASA’s next mission to Mars will be the topic of a media briefing Thursday, March 29, at JPL. The briefing will air live on NASA Television and the agency’s website.',
  'news_title': 'NASA Briefing on First Mission to Study Mars Interior'},
 {'news_p': "NASA spacecraft travel to far-off destinations in space, but a new mobile app produced by NASA's Jet Propulsion Laboratory, Pasadena, California, brings spacecraft to users.",
  'news_title': "New 'AR' Mobile App Features 3-D NASA Spacecraft"},
 {'news_p': "NASA invites digital creators to apply for social media credentials to cover the launch of the InSight mission to Mars, May 3-5, 

In [49]:
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import time
import pandas as pd

def init_browser():
        browser = Browser('chrome', headless=False)

def scrape_mars_two():
    browser = init_browser()
    url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
    response = requests.get(url)
    time.sleep(1)
    soup = BeautifulSoup(response.text, 'lxml')
    #create an empty dictionary to store values
    news_data={}
    #find the appropriate div class containing target data to scrape, scrape text and assign to variables
    news_p = soup.find_all('div',class_="rollover_description_inner")[0].text.strip()
    news_title = soup.find_all('div',class_="content_title")[0].text.strip()
    #put the variables into a dictionary
    post = {"news_title": news_title,
                    "news_p":news_p}
    print(post)
    #update the main dictionary with the new dictionary 
    news_data.update(post)
    browser = Browser('chrome', headless=False)
    #new URL to scrape 
    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)
    time.sleep(1)
    #use splinter to click on button 
    browser.click_link_by_partial_text('FULL IMAGE')
    time.sleep(1)
    #HTML Object 
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    #find the class where pic is stored
    results = soup.find(class_='fancybox-image')
    #retrieve source attribute, i.e. the path
    url = results['src']
    #attach the path to the main site link, this is the full image link
    featured_image_url = 'https://www.jpl.nasa.gov'+url
    #store the full image link in a dictionary
    post_two = {'featured_image':featured_image_url}
    #update the main dictionary with the new dictionary data
    news_data.update(post_two)
    print(post_two)
    #visit the mars twitter page to get the Weather
    url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(url)
    time.sleep(1)
    response = requests.get(url)
    #parse HTML with Beautiful soup, get the text
    soup = BeautifulSoup(response.text, 'html.parser')
    #get the text from the first p tag with appropriate class (from inspecting the site)
    mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
    #store the weather tweet in a dictionary
    post_three = {'mars_weather':mars_weather}
    #update the maind dictionary with the new dictionary data
    news_data.update(post_three)
    print(post_three)
    browser = Browser('chrome', headless=False)
    #new URL to be scraped
    url = 'https://space-facts.com/mars/'
    #read the table, put into list variable
    tables = pd.read_html(url)
    #convert the list to a dataframe
    mars_df =tables[0]
    #set column names 
    mars_df.columns = ["Characteristics","Values"]
    #set index to Characteristics
    mars_df = mars_df.set_index(["Characteristics"])
    #make it an html table
    mars_df.to_html("mars_data.html")
    #put the whole table into a dictionary item
    table_post={"mars_table":mars_df}
    #update the main dictionary with the table dictionary
    news_data.update(table_post) 
    print(table_post)
    browser = Browser('chrome', headless=False)
    #new URL to be scraped
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    time.sleep(1)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all(class_='item')
    #an empty array to store dictionary
    image_data=[]
    #loop through the item class
    for result in results:
        #find the first a tag
        link=result.find('a')
        #assign the href to variable 'links'
        links = link['href']
        #assign the link h3 title text to variable 'title'
        title =result.find('h3').text
        #concatenate the path with the main site link, assign to variable 'url'
        url='https://astrogeology.usgs.gov'+links
        #open brower, chromedriver
        browser = Browser('chrome', headless=False)
        #visit the concatenated url
        browser.visit(url)
        time.sleep(1)
        html = browser.html
        #parse the html with beautiful soup
        soup = BeautifulSoup(html, 'html.parser')
        #find all elemenst with class 'downloads', assign results to variable list 'infos'
        infos = soup.find_all(class_='downloads')
        
        for info in infos:
            link_two=info.find('a')
            img_url=link_two['href']
            post_four={'img_url':img_url, 'title':title}
            image_data.append(post_four)

     #update main dictionary with new data
    post_five={"image_data":image_data}
    print(post_five)
   
    news_data.update(post_five)
    return news_data
            


In [51]:
scrape_mars_two()

{'news_title': 'NASA Invests in Visionary Technology', 'news_p': 'NASA is investing in technology concepts, including several from JPL, that may one day be used for future space exploration missions.'}
{'featured_image': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18847_ip.jpg'}
{'mars_weather': 'Sol 2039 (May 02, 2018), Sunny, high 0C/32F, low -74C/-101F, pressure at 7.28 hPa, daylight 05:23-17:20'}
{'mars_table':                                              Values
Characteristics                                    
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.42 x 10^23 kg (10.7% Earth)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.52 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                  -153 to 20 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astron

{'featured_image': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18847_ip.jpg',
 'image_data': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
   'title': 'Cerberus Hemisphere Enhanced'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
   'title': 'Schiaparelli Hemisphere Enhanced'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
   'title': 'Syrtis Major Hemisphere Enhanced'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
   'title': 'Valles Marineris Hemisphere Enhanced'}],
 'mars_table':                                              Values
 Characteristics                                    
 Equatorial Diameter:                       6,792 km
 Polar Diameter:                            6,752 km
 Mass:            

In [17]:
for news in news_data:
    db.marsdata.insert_one(news)

In [25]:
db.marsdata.drop()

In [37]:
results = db.marsdata.find()
for result in results:
    print(result)